In [1]:
import sys
import pandas as pd
import os
from matplotlib import pyplot as plt
from scipy.stats import hmean
import tikzplotlib

dir_latency = "./paper/gzhm_variants_latency"
dir_thrput = "./paper/gzhm_external"
variants = next(os.walk(dir_latency))[1]
print(variants)

['TRHM', 'GZHM']


In [2]:
def humanize_nanoseconds(sec):
    time_str = ('%.2f ns') % (sec)
    if (sec > 1000):
        time_str = ('%.2f us') % (sec / 1000.0)
    if (sec > 1000_000):
        time_str = ('%.2f ms') % (sec / 1000_000.0)
    return time_str

def latency_distribution(dir, ops):
    summaries = pd.DataFrame()
    all_summaries = pd.DataFrame()
    for op in ops:
        for d in variants:
            df = pd.read_csv('./%s/%s/churn_latency.txt' % (dir, d), delim_whitespace=True)
            df = df.loc[(df["op"]==op)]
            if (len(df)==0):
                continue
            summaries[f'{d}'] = df['latency'].describe(percentiles=[.50, .90, .99, .9999])
        print(op)
        hsum = pd.DataFrame()
        for column in summaries.columns:
            hsum[column] = (summaries[column].map(lambda x : humanize_nanoseconds(x)))
        print(hsum.to_latex())
latency_distribution(dir_latency, ["DELETE", "INSERT", "LOOKUP"])

DELETE
\begin{tabular}{l}
\toprule
\midrule
\bottomrule
\end{tabular}

INSERT
\begin{tabular}{l}
\toprule
\midrule
\bottomrule
\end{tabular}

LOOKUP
\begin{tabular}{l}
\toprule
\midrule
\bottomrule
\end{tabular}



In [3]:
def humanize_bytes(bytes):
    bytes_str = ('%d B') % (bytes)
    if (bytes > 1024):
        bytes_str = ('%d KB') % (bytes / (1024))
    if (bytes > 1024 * 1024):
        bytes_str = ('%d MB') % (bytes / (1024* 1024))
    if (bytes > 1024 * 1024 * 1024):
        bytes_str = ('%d GB') % (bytes / (1024* 1024 * 1024))
    return bytes_str

def memory_usage(dir):    
    data = []
    labels = []
    df = pd.DataFrame()
    for d in variants:
        f_variant = open("%s/%s/test_params.txt" % (dir, d), "r")
        lines = f_variant.readlines()
        memory_usage = int(lines[0])
        data.append(memory_usage)
        labels.append(d)
    df['Hashmap'] = labels
    df['USAGE'] = data
    df['Size'] = df['USAGE'].map(lambda x: humanize_bytes(x))
    df['Space Efficiency'] = df['USAGE'].map(lambda x: (100 * x) / (0.95 * (2**22) * 16))
    print(df[['Hashmap', "Size", "Space Efficiency"]].to_latex(float_format="%.2f"))
memory_usage(dir_thrput)

FileNotFoundError: [Errno 2] No such file or directory: './paper/gzhm_external/TRHM/test_params.txt'